In [ ]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

In [ ]:
base_folder = "data"

# Fit Regression Model


## Input

* `samples_for_model.csv` - Observations 
    * user (int) - User identifier
    * week_index (int) - Week in the study period
    * count (int) - Number of items recommended to the user in this week
    * weeks_since_signup (int) - Number of weeks since the user's first interaction
    * variety (int) - Variety in topics or political

## Output

* `exp_coeff_table.csv` - Table with exponentiated coefficients (Intercept and weeks_since_signup)
* `other_coeff_table.csv` - Table with all coefficients as estimated by the model

In [ ]:
df = pd.read_csv(f"{base_folder}/samples_for_model.csv")

# Modeling

In [ ]:
# GLMM model configuration
formula = """
variety ~ 
1 
+ (weeks_since_signup) 
+ (np.log(count)) 
+ (1|user) 
+ (1|week_index)
""" 

In [ ]:
# Parameters for the experiment
draws = 1000
tune = 3000

In [ ]:
# 
model = bmb.Model(
    formula,
    df,
    family="negativebinomial",
    link="log",
    dropna=True,
    auto_scale=True
)


In [ ]:
trace = model.fit(draws=draws, tune=tune, target_accept=0.9)

In [ ]:
model.plot_priors()
plt.savefig(f'{base_folder}/plot_priors.pdf', dpi=300)

In [ ]:
az.plot_trace(trace, var_names=
              ["Intercept", "weeks_since_signup", "np.log(count)", "1|week_index", "1|week_index_sigma"])
plt.savefig(f'{base_folder}/plot_trace.pdf', dpi=300)


In [ ]:
exp_coeff_table = az.summary(
    np.exp(trace.posterior), var_names=[
        "Intercept",
        "weeks_since_signup", 
    ]
)


In [ ]:
exp_coeff_table

In [ ]:
exp_coeff_table.to_csv(f"{base_folder}/exp_coeff_table.csv", header=True, index=True)

In [ ]:
coeff_table = az.summary(trace.posterior, var_names=[
    "Intercept", 
    "weeks_since_signup", 
    "np.log(count)", 
    "1|week_index_sigma", 
    "1|user_sigma",
    "variety_alpha"
])

In [ ]:
coeff_table

In [ ]:
coeff_table.to_csv(f"{base_folder}/other_coeff_table.csv", header=True, index=False)